In [ ]:
import tensorflow as tf
import numpy as np
base_dir=r"C:\Users\Ishfaque\Micro_Train_Dataset"
IMAGE_SIZE=256
BATCH_SIZE=64

#pre=processing
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.1
    )

test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
     rescale=1./255,
     validation_split=0.1
)

train_datagen=train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'
)

test_datagen=test_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

cnn=tf.keras.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=64,padding='same',strides=2,kernel_size=3,activation='relu',input_shape=(256,256,3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',strides=2,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',strides=2,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(3,activation='softmax'))
cnn.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])
cnn.fit(train_datagen,epochs=20,validation_data=test_datagen)

In [ ]:
import serial
import MySQLdb
import time
import cv2 as cv2
import tensorflow as tf
import numpy as np
#function to detect an item
def detectItem():
    cam_port = 0
    cam = cv2.VideoCapture(cam_port)
   #cnn = joblib.load(Completed_model)

    result, image = cam.read()
    if result:
        # Resize the image
        dsize = (256, 256)
        output = cv2.resize(image, dsize)

        # Prepare the image for prediction
        img_pred = tf.keras.utils.img_to_array(output)
        img_pred = np.expand_dims(img_pred, axis=0)

        # Make the prediction
        rslt = cnn.predict(img_pred)
        print(rslt)

        # Determine the predicted object
        pred_num = max(rslt[0][0], rslt[0][1], rslt[0][2])
        if pred_num >0.80:     
            if pred_num == rslt[0][0]:
                prediction = "Humidity"
            elif pred_num == rslt[0][1]:
                prediction = "Bluetooth"
            else:
                prediction = "Transistor"
        else:
            prediction = "Unknown object"

        print(prediction)
        print(rslt[0][0])
        print(rslt[0][1])
        print(rslt[0][2])
    else:
        print("Error: Failed to capture image from the camera")
    return prediction



dbConn=MySQLdb.connect("localhost","root","","rfid_read") or die("Could not connect to DB")
cursor=dbConn.cursor()

device = 'COM9'

try:
    print("Trying...",device)
    arduino=serial.Serial(device,9600)
    
    
except: 
    print("Failed to connect on",device)
while True:
    time.sleep(1)
    
    try:
        
        data=arduino.readline()
        pieces=data.decode()
        print(pieces)
        try:
            cursor=dbConn.cursor();
            cursor.execute(""" SELECT * FROM user WHERE RFID LIKE %s """,(pieces,))
            row = cursor.fetchone()
            if row is None:
                print("It is not a valid ID Card")
                err = "  Invalid ID  "
                err = err + '\r'
                arduino.write(err.encode())
            else:
                #pieces = pieces+'\r'
                returnID = row[0]+'\r'
                arduino.write(returnID.encode())
                cursor.execute(""" INSERT INTO rfid_data (ID, Member_ID) VALUES (NULL,%s) """,(pieces,))
                dbConn.commit()
                orderID = int(cursor.lastrowid);
                # rfid has been inserted, check if there are any outstanding orders (orders that have not been returned yet in 7 days)
                # SELECT ID FROM `rfid_data` where Member_ID='(rfid)' and state='0' and DATEDIFF(CURRENT_DATE,date)>7;
                # returns the order ids of the outstanding orders
                
                while True:
                    # loop has to exist here.
                    while(arduino.inWaiting()==0):
                        pass
                    opt_data = arduino.readline()
                    option = opt_data.decode()
#                     opt_data = str(opt_data,'uft-8')
#                     opt_data = opt_data.strip('\r\n')
                    print("option: "+str(option))
                    # 1 : borrow  2 : return # : exit
                    option = option.strip('\r\n')
                    if option == "1":
                        # borrow item
                        while(arduino.inWaiting()==0):
                            pass
                        item_data = arduino.readline()
#                         brw_data = str(brw_data,'uft-8')
#                         brw_data = brw_data.strip('\r\n')
                        item_data = item_data.decode()
                        print("item = "+str(item_data))
                        
                        item = int(item_data)
                        if item<=3:
                            while(arduino.inWaiting()==0):
                                pass
                            qty_data = arduino.readline()
                            qty_data = qty_data.decode()
                            quantity = int(qty_data)

                            # sql = "SELECT stock FROM `inventory` WHERE id=item;"
                            cursor.execute("SELECT stock FROM `inventory` WHERE id = %s",(item,))
                            stock = cursor.fetchone()
                            print(str(stock[0]))
                            if (stock[0]-quantity)>=0:

                                msg_ok = "ok"
                                msg_ok = str(msg_ok) + '\r'
                                arduino.write(msg_ok.encode())

                                # sql = "INSERT INTO `orders` (`ID`, `item_id`, `quantity`) VALUES ('"+orderID+"', '"+item+"', '"+quantity+"')"
                                cursor.execute("INSERT INTO `orders` (`ID`, `item_id`, `quantity`) VALUES ('%s', '%s', '%s')",(orderID,item,quantity,))
                                # INSERT INTO `orders` (`ID`, `item_id`, `quantity`) VALUES ('(orderID)', '(item)', '(quantity)');
                                dbConn.commit()
                                #update stock on inventory
                                #update inventory join orders on (inventory.ID=orders.item_id) set stock=stock-orders.quantity where orders.ID='(orderID)'
                                cursor.execute("update inventory join orders on (inventory.ID=orders.item_id) set stock=stock-orders.quantity where orders.ID='%s' AND inventory.ID='%s'",(orderID,item))
                                dbConn.commit()
                                #update order to state = 0
                                #UPDATE `rfid_data` SET `state` = '0' WHERE `rfid_data`.`ID` = (orderID);
                                cursor.execute("UPDATE `rfid_data` SET `state` = '0' WHERE `rfid_data`.`ID` ='%s'",(orderID,))
                                dbConn.commit()
                            else:
                                stk_ch = stock[0]
                                stk_ch = str(stk_ch) + '\r'
                                arduino.write(stk_ch.encode())
                        
                    elif option == "2":
                        # return item
                        while(arduino.inWaiting()==0):
                            pass
                        item_placed = arduino.readline()
                        item_placed = item_placed.decode()
                        item_placed = item_placed.strip('\r\n')
                        if item_placed == "*":
                            print(item_placed)
                            #item is recognised here
                            prediction = detectItem()
                            #print(prediction)
                            query = "update inventory set stock=stock+1 where inventory.item='"+prediction+"'"
                            #print(query) 
                            prediction = prediction + '\r'
                            arduino.write(prediction.encode())
                            
                            servo_msg = arduino.readline()
                            servo_msg = servo_msg.decode()
                            servo_msg = servo_msg.strip('\r\n')
                            print(servo_msg)
                            #update stock on inventory
                            #update inventory join orders on (inventory.ID=orders.item_id) set stock=stock+orders.quantity where orders.ID='(order id)'
                            #cursor.execute("""update inventory set stock=stock+1 where inventory.item='%s'""",(prediction,))
                            cursor.execute(query)
                            dbConn.commit()
                            #update order to state = 1
                            #UPDATE `rfid_data` SET `state` = '1' WHERE `rfid_data`.`ID` = (orderID);
                            cursor.execute("UPDATE `rfid_data` SET `state` = '1' WHERE `rfid_data`.`ID` ='%s'",(orderID,))
                            dbConn.commit()
                    elif option == "#":
                        #cancel
                        # DELETE FROM rfid_data WHERE `rfid_data`.`ID` = (order) and state=NULL
                        #maybe first check if state==NULL then delete
#                         cursor.execute("DELETE FROM rfid_data WHERE `rfid_data`.`ID` = '%s' and state=NULL",(orderID,))
#                         dbConn.commit()
                        break
                
                
            cursor.close()
        except MySQLdb.IntegrityError:
            print("failed to insert data");
        finally:
            cursor.close()
    except Exception as e:
        print(e)
        print("Processing")
    